In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import csv
from functions_simulator import *
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

# To change the numerotation of the participants for which we want to plot numerical results
def numerotation_part(x):
    part = ['307','308','309','301','303','304','364','365','366','367']
    for i in range(len(part)):
       # print(part[i])
       # print(x == part[i])
        if x == part[i]:
            return str(i+1)
    print('add participant')

Save of simulation of times for some participants with median parameters given by SBI

In [ ]:
#participants for which we want to save times simulations
participants = ['307','308','309','301','303','304','364','365','366','367']

eta_inf = 0.09
eta_sup = 2
s_inf = 0.04
s_sup = 1/5

for part in participants:

    sample = torch.load(f'/Users/sophiejaffard/Desktop/Expé/old code discrete time/saves_sbi/sample_new_{eta_inf}_{eta_sup}_{s_inf}_{s_sup}_{part}_{part}.pt')

    eta_sim = sample.numpy()[:,0]
    s_sim = sample.numpy()[:,1]

    N = np.size(eta_sim)

    eta_med = eta_sim[int(N/2)]
    eta_moy = np.sum(eta_sim) / N

    s_med = s_sim[int(N/2)]
    s_moy = np.sum(s_sim) / N

    N_simu = 100

    print(eta_med, eta_moy, s_med, s_moy)

    nb_obj_max = 100

    times_str = []

    with open(f'{part}.csv', 'r') as file:
        reader = csv.reader(file, delimiter=',')
        for row in reader:
                #print(row)
            if row[4] == 'timeout':
                times_str.append('5')
            else:
                times_str.append(row[6])
            #print(times_str)

    nb = len(times_str)
    times_list = [float(times_str[i]) for i in range(1,nb)]
    n = len(times_list)
    nb_obj_learning = n - 18 #obj before transfer

   
    times = torch.tensor(times_list)
    
    #print(times_0[:n])
    #print(times[:nb_iter])

    times_simu = np.zeros((n, N_simu))
    print(nb_obj_learning)
    print(n)
    for i in range(N_simu):
        times, accuracy = simulator_vectorized_fix_nb_obj(eta_med, s_med, nb_obj_learning)
        times_simu[:,i] = times

    #save the simulation
    np.save(f"cum_times_{part}", times_simu)

Plot of cumulated times and median simulated cumulated times for some participants

In [ ]:
#Sample data for testing

participants = ['301','303','304','364', '365', '366']

n_part = len(participants)
fig, axs = plt.subplots(2, 3, figsize=(10, 6))
N_simu = 100
for i in range(n_part):
    part = participants[i]
    times_str = []
    try:
        times_simu = np.load(f"cum_times_{part}.npy")
        
        with open(f'/Users/sophiejaffard/Desktop/Expé/all data/{part}.csv', 'r') as file:
            reader = csv.reader(file, delimiter=',')
            for row in reader:
                    #print(row)
                if row[4] == 'timeout':
                    times_str.append('5')
                else:
                    times_str.append(row[6])
                #print(times_str)

        nb = len(times_str)
        n = nb-1
        times_list = [float(times_str[l]) for l in range(1,nb)]
        times_0 = np.array(times_list)

        times_moy = np.sum(times_simu, axis = 1) / N_simu
        times_med = np.sort(times_simu, axis = 1)[:,int(N_simu / 2)]
        times_10 = np.sort(times_simu, axis = 1)[:,int(N_simu*0.1)]
        times_90 = np.sort(times_simu, axis = 1)[:,int(N_simu*0.9)]

        cum_part = np.cumsum(times_0[:n])
        cum_hat = np.cumsum(times_med[:n])

        X1 = np.arange(n)
        if i <= 2:
            x=0
            y = i
        else:
            x = 1
            y = i - 3

        axs[x,y].plot(X1,cum_part, label = 'participant')
        axs[x,y].plot(X1, cum_hat, label = 'median')
        #axs[i].set_ylim(0,120)
        #axs[i].set_xlim(0,57)
        axs[x,y].set_xlabel('Number of answers')
        axs[x,y].set_ylabel('Cumulated answer times')
        axs[x,y].fill_between(X1, np.cumsum(times_10[:n]), np.cumsum(times_90[:n]), color='orange', alpha=.2)
        #axs[x,y].set_title(f'Participant {part}')
        #print(part)
        axs[x,y].set_title(f'Participant {numerotation_part(part)}')
        axs[x,y].legend()

    except FileNotFoundError:
        print(f"File for sample {i + 1} not found. Please check the path and filename.")

plt.tight_layout()
#plt.savefig('cum_times_numero.pdf')
plt.show()

Plot of the answer ticks 

In [ ]:
#participants = ['301','303','304']
participants = ['303','364', '367']

eta_inf = 0.09
eta_sup = 2
s_inf = 0.04
s_sup = 1/5

nb_obj_max = 100

n_simu = 1 #number of examples of simulated data

n_part = len(participants)
fig, axs = plt.subplots(n_simu + 1, n_part, figsize=(10, 5)) 

for i in range(n_part):
    part = participants[i]
    times_str = []
    try:
        #times_simu = np.load(f"/Users/sophiejaffard/Desktop/Expé/saves_sbi/cum_times_{part}.npy")
        correct_answers = [] 
        
        with open(f'{part}.csv', 'r') as file:
            reader = csv.reader(file, delimiter=',')
            for row in reader:
                    #print(row)
                if row[4] == 'timeout':
                    times_str.append('5')
                else:
                    times_str.append(row[6])
                if row[5]=='True':
                    correct_answers.append(1)
                   
                if row[5]=='False':
                    correct_answers.append(0)
                
        nb = len(times_str) - 1
        n_learning = nb - 18 - 15 
        nb_obj_learning = nb - 18
        times_list = [float(times_str[l]) for l in range(1,nb)]
        times_0 = np.array(times_list)[:nb_obj_learning]
        cum_times = np.cumsum(times_0)
       
        X1 = np.arange(nb_obj_learning)
        
        
        axs[0,i].eventplot(cum_times, lineoffsets=1, linelengths=0.9)
        axs[0,i].set_yticks([])


        #axs[0,i].set_title(f'Participant {part}')
        axs[0,i].set_title(f'Participant {numerotation_part(part)}')
        #axs[0,i].legend()

        sample = torch.load(f'sample_new_{eta_inf}_{eta_sup}_{s_inf}_{s_sup}_{part}_{part}.pt')

        #find median parameters
        eta_sim = sample.numpy()[:,0]
        s_sim = sample.numpy()[:,1]
        N = np.size(eta_sim)
        eta_med = eta_sim[int(N/2)]
        s_med = s_sim[int(N/2)]

        for j in range(1,n_simu+1):
            #simulate the model
            
            times, accuracy = simulator_vectorized_fix_nb_obj(eta_med, s_med, nb_obj_learning)
            
        
            times_0 = times[:nb_obj_learning]
            cum_times = np.cumsum(times_0)

            
            axs[j,i].eventplot(cum_times, lineoffsets=1, linelengths=0.9)

            axs[j,i].set_title('Simulation with $\hat{\eta}$ and $\hat{s}$')
            axs[j,i].set_xlabel('Time (s)')
            axs[j,i].set_yticks([])
            #axs[j,i].set_ylim(y_inf[i])
            #axs[j,i].legend()

    except FileNotFoundError:
        print(f"File for sample {i + 1} not found. Please check the path and filename.")

plt.tight_layout()
#plt.savefig('cum_answers_num.pdf')
plt.show()

# The wrong answers were added manually using the lists 'correct_answers' and 'accuracy'.
            

Group analysis

In [ ]:
part_middle_school= ['416','439','443','448','455','458','459','461','463','465','478','483','301', '302', '303', '304','307', '308', '309', '310', '311','312','314','316','315','722','737','738','739','740','741','749','753','759','765','769','772','773','774','777','780','783','784','787','790']
part_university = ['339','355','405','406','363', '364', '365', '366', '367', '368', '369', '372', '375', '376', '377', '380', '381','385', '386', '388', '390', '392', '396', '397', '400', '404','408', '695','668','666','659','655','653','652','643','640','550','551','554','555','561','563','564','565','577','580','583','586','587','588', '597','601','605','608']
tot_participants = part_middle_school + part_university
n_middle_school = len(part_middle_school)
n_university = len(part_university)
n_tot = len(tot_participants)

eta_inf = 0.09
eta_sup = 2
s_inf = 0.04
s_sup = 1/5


eta_med = np.zeros(n_middle_school + n_university)
eta_moy = np.zeros(n_middle_school + n_university)
s_med = np.zeros(n_middle_school + n_university)
s_moy = np.zeros(n_middle_school + n_university)
titles = ['With every participant','Middle school students','Undergraduate students']

for i in range(n_tot):
    part = tot_participants[i]
    sample = torch.load(f'sample_new_{eta_inf}_{eta_sup}_{s_inf}_{s_sup}_{part}_{part}.pt')

    eta_sim = sample.numpy()[:,0]
    s_sim = sample.numpy()[:,1]

    N = np.size(eta_sim)

    eta_med[i] = eta_sim[int(N/2)]
    eta_moy[i] = np.sum(eta_sim) / N

    s_med[i] = s_sim[int(N/2)]
    s_moy[i] = np.sum(s_sim) / N

fig, axs = plt.subplots(2, 3, figsize=(8, 5)) 
axs[0, 0].get_shared_y_axes().join(axs[0, 0], axs[0, 1], axs[0,2])
bins = np.array([15,15,15])

for j in range(3):
    if j == 0:
        eta_med0 = eta_med
        eta_moy0 = eta_moy
        s_med0 = s_med
        s_moy0=s_moy
    elif j == 1:
        eta_med0 = eta_med[:n_middle_school]
        eta_mpy0 = eta_moy[:n_middle_school]
        s_med0 = s_med[:n_middle_school]
        s_moy0=s_moy[:n_middle_school]
    elif j == 2:
        eta_med0 = eta_med[n_middle_school:]
        eta_mpy0 = eta_moy[n_middle_school:]
        s_med0 = s_med[n_middle_school:]
        s_moy0=s_moy[n_middle_school:]
    
    axs[0,j].hist(eta_med0, label = 'Median $\hat{\eta}$', bins = bins[j],density = True)
    axs[0,j].set_title(titles[j])
    axs[0,j].legend()

    axs[1,j].hist(s_med0, label = r'Median $\hat{\theta}$', bins = bins[j], density = True)
    axs[1,j].legend()

    #fig.suptitle(titles[j])
plt.tight_layout()
#plt.savefig('hist_groups.pdf')
plt.show()
